# Improvise a Jazz Solo with an LSTM Network

- Apply an LSTM to music generation.
- Generate jazz music with deep learning.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp -R gdrive/My\ Drive/Colab\ Notebooks/DeepLearning.ai/W1JazzImprovisationLSTM/* .

In [0]:
!cp -R gdrive/My\ Drive/Colab\ Notebooks/DeepLearning.ai/W1JazzImprovisationLSTM/ImproviseJazzWithLSTM* .

In [0]:
!pip install music21

In [4]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


## 1 - Model & Data

### 1.1 - Dataset

You will train your algorithm on a corpus of Jazz music. Run the cell below to listen to a snippet of the audio from the training set:

In [7]:
IPython.display.Audio('./data/training_example.mp3')

In [19]:
X, Y, n_values, indices_values = load_music_utils()
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

shape of X: (60, 30, 78)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 78
Shape of Y: (30, 60, 78)


- `X`: This is an (m, $T_x$, 78) dimensional array. We have m training examples, each of which is a snippet of $T_x =30$ musical values. At each time step, the input is one of 78 different possible values, represented as a one-hot vector. Thus for example, X[i,t,:] is a one-hot vector representating the value of the i-th example at time t. 

- `Y`: This is essentially the same as `X`, but shifted one step to the left (to the past). Similar to the dinosaurus assignment, we're interested in the network using the previous values to predict the next value, so our sequence model will try to predict $y^{\langle t \rangle}$ given $x^{\langle 1\rangle}, \ldots, x^{\langle t \rangle}$. However, the data in `Y` is reordered to be dimension $(T_y, m, 78)$, where $T_y = T_x$. This format makes it more convenient to feed to the LSTM later. 

- `n_values`: The number of unique values in this dataset. This should be 78. 

- `indices_values`: python dictionary mapping from 0-77 to musical values.

### 1.2 - Overview of the model

LSTM model implemented using Keras. The architecture is as follows: 

<img src="images/music_generation.png" style="width:600;height:400px;">

<!--
<img src="images/djmodel.png" style="width:600;height:400px;">
<br>
<caption><center> **Figure 1**: LSTM model. $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ is a window of size $T_x$ scanned over the musical corpus. Each $x^{\langle t \rangle}$ is an index corresponding to a value (ex: "A,0.250,< m2,P-4 >") while $\hat{y}$ is the prediction for the next value  </center></caption>
!--> 


*   Train the model on random snippets of 30 values taken from a much longer piece of music. 
*   Don't bother to set the first input $x^{\langle 1 \rangle} = \vec{0}$, since now most of these snippets of audio start somewhere in the middle of a piece of music. 
*   Set each of the snippets to have the same length $T_x = 30$ to make vectorization easier. 




## 2 - Building the model

Build a model that takes in X of shape $(m, T_x, 78)$ and Y of shape $(T_y, m, 78)$. 

Use an LSTM with 64 dimensional hidden states. Lets set `n_a = 64`. 


In [0]:
n_a = 64 

Define the layers objects as global variables to use in dj model. [Reshape()](https://keras.io/layers/core/#reshape), [LSTM()](https://keras.io/layers/recurrent/#lstm), [Dense()](https://keras.io/layers/core/#dense).


In [0]:
reshapor = Reshape((1, 78))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

Implement `djmodel()`. 
1. Create an empty list "outputs" to save the outputs of the LSTM Cell at every time step.
2. Loop for $t \in 1, \ldots, T_x$:

    A. Select the "t"th time-step vector from X. The shape of this selection should be (78,). To do that, create a custom [Lambda](https://keras.io/layers/core/#lambda) layer in Keras 

    B. Reshape x to be (1,78).

    C. Run x through one step of LSTM_cell. Remember to initialize the LSTM_cell with the previous step's hidden state $a$ and cell state $c$.

    D. Propagate the LSTM's output activation value through a dense+softmax layer using `densor`. 
    
    E. Append the predicted value to the list of "outputs"
 


In [0]:
# djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs= [X, a0, c0], outputs=outputs)
    
   
    
    return model

Define the model. Use `Tx=30`, `n_a=64` (the dimension of the LSTM activations), and `n_values=78`.

In [0]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

Compile the model to be trained. Use Adam optimizer and a categorical cross-entropy loss.

In [0]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Initialize `a0` and `c0` for the LSTM's initial state to be zero. 

In [0]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

Fit the model.

Turn `Y` to a list since the cost function expects `Y` to be provided in this format (one list item per time-step). So `list(Y)` is a list with 30 items, where each of the list items is of shape (60,78).



In [26]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 26s 431ms/step - loss: 125.9099 - dense_2_loss: 0.0000e+00 - dense_2_acc: 0.0500 - dense_2_acc_1: 0.0000e+00 - dense_2_acc_2: 0.0833 - dense_2_acc_3: 0.0167 - dense_2_acc_4: 0.0500 - dense_2_acc_5: 0.0833 - dense_2_acc_6: 0.0833 - dense_2_acc_7: 0.0500 - dense_2_acc_8: 0.0333 - dense_2_acc_9: 0.0833 - dense_2_acc_10: 0.0667 - dense_2_acc_11: 0.0667 - dense_2_acc_12: 0.0333 - dense_2_acc_13: 0.1167 - dense_2_acc_14: 0.0667 - dense_2_acc_15: 0.0500 - dense_2_acc_16: 0.0333 - dense_2_acc_17: 0.0167 - dense_2_acc_18: 0.1000 - dense_2_acc_19: 0.0333 - dense_2_acc_20: 0.0500 - dense_2_acc_21: 0.0333 - dense_2_acc_22: 0.0667 - dense_2_acc_23: 0.1167 - dense_2_acc_24: 0.0667 - dense_2_acc_25: 0.1500 - dense_2_acc_26: 0.0833 - dense_2_acc_27: 0.0667 - dense_2_acc_28: 0.1167 - dense_2_acc_29: 0.0000e+00                                                                          
Epoch 2/100
60/60 [==============================] - 0s 2ms/step - l

## 3 - Generating music

The trained model has learned the patterns of the training dataset music. Now use this model to synthesize new music. 

#### 3.1 - Predicting & Sampling

At each step of sampling, take as input the activation `a` and cell state `c` from the previous state of the LSTM, forward propagate by one step, and get a new output activation as well as cell state. The new activation `a` can then be used to generate the output, using `densor` as before. 

First, initialize `x0` as well as the LSTM activation and and cell value `a0` and `c0` to be zeros. 


[Minor technical note: Rather than sampling a value at random according to the probabilities in `out`, this line of code actually chooses the single most likely note at each step using an argmax.]


In [0]:
# music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

   
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    
    
    
    return inference_model

Define the inference model. Generate 50 values (Ty).

In [0]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

The zero-valued vectors to initialize `x` and the LSTM state variables `a` and `c`. 

In [0]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

Implement `predict_and_sample()`. 
1. Use the inference model to predict an output given the set of inputs. The output `pred` is a list of length $T_y$ where each element is a numpy-array of shape (1, n_values).
2. Convert `pred` into a numpy array of $T_y$ indices. Each index corresponds is computed by taking the `argmax` of an element of the `pred`.
3. Convert the indices into their one-hot vector representations.

In [0]:
# predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices, num_classes=78)
   
    
    return results, indices

In [32]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

np.argmax(results[12]) = 55
np.argmax(results[17]) = 52
list(indices[12:18]) = [array([55]), array([33]), array([55]), array([23]), array([46]), array([52])]


#### 3.3 - Generate music 

In [33]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 51 sounds using the predicted values for the set of chords ("1") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("2") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("4") and after pruning
Generated 51 sounds using the predicted values for the set of chords ("5") and after pruning
Your generated music is saved in output/my_music.midi


In [0]:
IPython.display.Audio('./data/30s_trained_model.mp3')

### Github

In [0]:
!git config --global user.name "username"
!git config --global user.email "ronlee94@g.ucla.edu"

In [0]:
!git add data/* images/* output/* *.py *.ipynb *.midi

In [0]:
!git commit -m "test"

In [0]:
!git remote add origin git@github.com:ronhlee/JazzImprovisation.git

In [0]:
!git push --set-upstream origin master